In [1]:
from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'scroll': True,
        'width': '100%'
})

from IPython.core.display import HTML
import urllib2
HTML("""
<style>
.reveal pre code {
   max-height: 1000px;
   font-size: 80%;
   line-height: 100%;
}
</style>
""")

## Introduction to Django

<sub>by Joe Meilinger</sub>


## Why use Django?

- **GREAT** community support 
- Used by industry giants: Disqus, Instagram, Pinterest, The Onion, NASA, etc
- Weathered the changes in industry over time, first public release back in 2005
- Variety of community-supported plugins/modules available, see <https://djangopackages.org/>

## Batteries included approach

- Opionated, MVC-ish oriented
- Relational database backend required
- Built in testing support
- Routing engine
- Powerful template processing
- Built-in support for authentation (via `contrib` modules) as well as GIS features
- Pairs very well with PostGIS (for GIS)

## Things we'll be covering in this talk

- Installing and getting started
- Models
- Python REPL
- Django Administration
- Views
- Templates
- Routing

<div class="text-center">There is a **ton** more to Django!</div>

## The Example

Very, very simple structure for being able to track the location of different types of trees.

## Installing Django

```bash
pip install django
pip install psycopg2  # for postgres support
pip install ipython  # for more friendly REPL
```

## Install PostgreSQL

- Mac: https://postgresapp.com
- Windows: https://www.postgresql.org/download/windows/

```bash
psql -c "create database treeseeker"
psql treeseeker -c "create extension postgis"
```

## Install GDAL

- Mac: https://sandbox.idre.ucla.edu/sandbox/general/how-to-install-and-run-gdal OR `brew install gdal`
- Windows: https://pythongisandstuff.wordpress.com/2016/04/13/installing-gdal-ogr-for-python-on-windows/

## Create a barebones project

In Django a "project" is our main application container that orchestrates "apps" contained within.

```bash
django-admin startproject treeseeker  # Create Django project
cd treeseeker
python manage.py startapp core  # House for our models/views/templates
```

Lets enable our app in the settings for our project (`treeseeker/settings.py`):

```python
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'django.contrib.gis',  # <-- needed for GIS capabilities
    'core'  # <-- our application
]
```

While we are in there...lets wire up our Postgres database connection:

```python
DATABASES = {
    'default': {
        'ENGINE': 'django.contrib.gis.db.backends.postgis',
        'NAME': 'treeseeker'
    }
}
```

## Model/Template/View/Routes

**Models:** contain entities (mostly persistent)

**Views:** manipulate models, handle passing data to templates

**Templates:** render HTML/visualize the models

**Route:** direct the incoming requests to views based on URL structure

## Great!  What's next

Completely based on user preference, but usually start with models:

- Flesh out the persistent components of the site first
- Using the Python interpreter allows for easy object creation and testing => `python manage.py shell`

## Models

`TreeSpecies` describes a single tree species

`TreeSample` describes a single sample of a tree


In `core/models.py`:

```python
from django.contrib.gis.db import models  # We're importing from gis since we're using a geometry

class TreeSpecies(models.Model):
    """ Describes a single species of tree """
    common_name = models.CharField(max_length=1024, null=True, blank=True)
    genus = models.CharField(max_length=255)
    species = models.CharField(max_length=255)

    def __str__(self):
        return '({} {}) {}'.format(self.genus, self.species, self.common_name)

    @staticmethod
    def from_csv(fname):
        """ Loads species information from a CSV (as provided from www.theplantlist.org) """
        import csv
        ret = []
        reader = csv.DictReader(open(fname))
        for r in reader:
            ret.append(TreeSpecies(genus=r['Genus'], species=r['Species']))
        return ret

class TreeSample(models.Model):
    """ Describes a single sample/observation of a tree """
    objects = models.GeoManager()
    geom = models.PointField()  # The physical lat/lon of the tree
    treespecies = models.ForeignKey(TreeSpecies, on_delete=models.CASCADE)
    height = models.DecimalField(null=True, blank=True, max_digits=5, decimal_places=2)
    crown_width = models.DecimalField(null=True, blank=True, max_digits=5, decimal_places=2)
    collected_at = models.DateTimeField(auto_now_add=True)
```

## The Python read eval print loop (REPL)

```bash
python manage.py shell
```

Let's load some data!

```bash
> from core.models import TreeSpecies
> trees = TreeSpecies.from_csv('../Fageceae.csv')  # Only loads into memory, not yet saved!
> for t in trees:
>     t.save()

ProgrammingError: relation "core_treespecies" does not exist
```

What?!?!

## Migrations -- versioning your database

Django can automatically create migrations for you from your declared models!

```bash
python manage.py makemigrations
```

See `core/migrations/0001_initial.py`.  Now we can modify our local development database via:

```bash
python manage.py migrate
python manage.py createsuperuser  # Lets create a superuser right now (for a future step)
```

## Running locally

```bash
python manage.py runserver
```

Open browser to <http://localhost:8000>, automatically reloads when code changes

## Adminstration for free!

In `core/admin.py`:

```python
from django.contrib.gis import admin
from core.models import TreeSample, TreeSpecies

class TreeSampleAdmin(admin.OSMGeoAdmin):
    list_display = ['treespecies', 'collected_at']
    search_fields = ['treespecies__genus', 'treespecies__species', 'treespecies__common_name']

class TreeSpeciesAdmin(admin.OSMGeoAdmin):
    list_display = ['genus', 'species', 'common_name']
    search_fields = ['genus', 'species', 'common_name']

admin.site.register(TreeSample, TreeSampleAdmin)
admin.site.register(TreeSpecies, TreeSpeciesAdmin)
```

## Views

In `core/views.py`:

```python
from django.shortcuts import render
from django.views.generic import ListView, DetailView
from core.models import TreeSpecies, TreeSample

# Create your views here.

class TreeSpeciesList(ListView):
    model = TreeSpecies
    template_name = 'index.html'

class TreeSampleDetail(DetailView):
    model = TreeSample
```

## Templates

In `core/templates/index.html`:

```python
<!DOCTYPE html>
<html lang="en">
  <body>
    {% for species in object_list %}
      {{ species }} - {{ species.treesample_set.count }}<br/>
    {% empty %}
      No species!
    {% endfor %}
  </body>
</html>
```

## Routing

In `core/urls.py`:

```python
from django.conf.urls import url, include
from django.contrib import admin
from .views import TreeSpeciesList

urlpatterns = [
    url(r'^$', TreeSpeciesList.as_view(), name='species-list'),
]
```

In `treeseeker/urls.py`:

```python
from django.conf.urls import url, include
from django.contrib import admin

urlpatterns = [
    url(r'^', include('core.urls')),
    url(r'^admin/', admin.site.urls),
]
```

## Next steps

- Django Forms
- Unit testing

## Plugins to check out

- API framework for Django: http://www.django-rest-framework.org/
- General Django extensions: http://django-extensions.readthedocs.io/en/latest/ (lots of gems)
- Debugging (in browser): http://django-debug-toolbar.readthedocs.io/en/latest/

## Resources

- https://www.djangoproject.com/
- https://djangopackages.org/

## Thank you!

**@joemeilinger** on Twitter<br/>
joemeilinger@gmail.com<br/>
**meilinger** on STL Tech Slack organization => get your invite at https://stltech.herokuapp.com

Also, if you're looking for some education and path to a career in Python, be sure to check out https://launchcode.org/apply